# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from Config.api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
file = '../Outputs/city_weather.csv'
df = pd.read_csv(file)

df.head()


,Unnamed: 0,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind_Speed,Country,Date
0,0,lahad datu,5.0268,118.3270,24.01,93,100,1.42,MY,1626295631
1,1,karachi,24.9056,67.0822,28.90,84,75,4.63,PK,1626295516
2,2,ushuaia,-54.8000,-68.3000,-0.19,80,75,2.06,AR,1626295610
3,3,kultuk,51.7208,103.6956,13.17,91,2,2.64,RU,1626295632
4,4,penalva,-3.2942,-45.1736,27.04,57,3,1.14,BR,1626295632


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
loc = df[['Latitude','Latitude']]

fig = gmaps.figure()
lay = gmaps.heatmap_layer(loc, df.Humidity)

fig.add_layer(lay)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:

# Pulled data in metric because freedom units are aweful and should be divisible by 10. So i did the proper conversions outside notebook.
df_filtered = df.loc[(df['Max Temp'] < 26.7) & (df['Max Temp'] > 21.1) & (df.Wind_Speed < 4.47) & (df.Cloudiness == 0)].dropna()



df_filtered.head()

,Unnamed: 0,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind_Speed,Country,Date
9,9,rikitea,-23.1203,-134.9692,22.81,71,0,3.99,PF,1626295391
36,36,rikitea,-23.1203,-134.9692,22.81,71,0,3.99,PF,1626295391
38,38,alexandria,31.2156,29.9553,26.63,64,0,3.73,EG,1626295639
64,64,marsa matruh,31.3525,27.2453,26.11,75,0,3.97,EG,1626295644
90,90,rikitea,-23.1203,-134.9692,22.81,71,0,3.99,PF,1626295650


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = df_filtered[['City','Country','Latitude','Longitude']].copy()
hotel_df['Hotel_Name'] = ''
hotel_df

,City,Country,Latitude,Longitude,Hotel_Name
9,rikitea,PF,-23.1203,-134.9692,
36,rikitea,PF,-23.1203,-134.9692,
38,alexandria,EG,31.2156,29.9553,
64,marsa matruh,EG,31.3525,27.2453,
90,rikitea,PF,-23.1203,-134.9692,
122,rikitea,PF,-23.1203,-134.9692,
142,rikitea,PF,-23.1203,-134.9692,
184,rikitea,PF,-23.1203,-134.9692,
192,rikitea,PF,-23.1203,-134.9692,
240,rikitea,PF,-23.1203,-134.9692,


In [7]:
# Heads Up used solutions for help on this part!

# Set parameters to search for a hotel
parameters = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# Iterate through 
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]
    
    parameters["location"] = f"{lat},{lng}"
        
    # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    response = requests.get(base_url, params=parameters)
    
    data = response.json()
    
    # Grab the first hotel from the results and store the name
    try:
        hotel_df.loc[index, "Hotel Name"] = data["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

hotel_df

,City,Country,Latitude,Longitude,Hotel_Name,Hotel Name
9,rikitea,PF,-23.1203,-134.9692,,People ThankYou
36,rikitea,PF,-23.1203,-134.9692,,People ThankYou
38,alexandria,EG,31.2156,29.9553,,Royal Crown Hotel
64,marsa matruh,EG,31.3525,27.2453,,Beau Site
90,rikitea,PF,-23.1203,-134.9692,,People ThankYou
122,rikitea,PF,-23.1203,-134.9692,,People ThankYou
142,rikitea,PF,-23.1203,-134.9692,,People ThankYou
184,rikitea,PF,-23.1203,-134.9692,,People ThankYou
192,rikitea,PF,-23.1203,-134.9692,,People ThankYou
240,rikitea,PF,-23.1203,-134.9692,,People ThankYou


In [19]:
print(data)

{'html_attributions': [], 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': -23.1206197, 'lng': -134.9687529}, 'viewport': {'northeast': {'lat': -23.1194598197085, 'lng': -134.9673886697085}, 'southwest': {'lat': -23.1221577802915, 'lng': -134.9700866302915}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'name': 'People ThankYou', 'opening_hours': {'open_now': True}, 'photos': [{'height': 2304, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/111098332724302207930">luigia b.</a>'], 'photo_reference': 'Aap_uEAoDIMtBCbsCkfv_hr9Rk9cLWNcBCJ_Dl6o4dhaXHrJHg5LCVb1TF7-PryXGSVxacQrnWj5OFCdedqFOfnyuil_KRUGEgqpABuwSmBqQv4LrBoW_2bEldxHVNqub2jXb_x-1LIyoFtPSD-BTrEb_gPX6lLWMkxkaQbj2QIGXtzfHAcM', 'width': 4096}], 'place_id': 'ChIJnW6Jh0aIJ54RV2HnC1r0Hjw', 'plus_code': {'global_code': '5487V2HJ+QF'}, 'rating': 4, 'reference': 'ChIJnW6Jh0aIJ54RV2HnC1r0Hjw', 'scope': 'GOOGLE', 'types': ['lodging', 'point_of_intere

In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [10]:
# Add marker layer ontop of heat map
lay2 = gmaps.marker_layer(locations)
fig.add_layer(lay2)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))